In [13]:
# 필요한 라이브러리 설치
%pip install langchain langchain-community langchain-openai chromadb openai pypdf

  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.6.1
    Not uninstalling urllib3 at /opt/jhub-venv/lib/python3.12/site-packages, outside environment /home/sohee/myenv
    Can't uninstall 'urllib3'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import glob

# 현재 위치 확인
os.chdir('/home/sohee/Bidding-Mate')
print(f"현재 작업 디렉토리: {os.getcwd()}")

# PDF 파일들 찾기 (Bidding-Mate 빼기!)
pdf_files = glob.glob('team_members/sohee/sample_pdfs/*.pdf')
print(f"✅ 총 PDF 파일 개수: {len(pdf_files)}")

# 처음 5개 파일명 확인
print("\n처음 5개 파일:")
for i, file in enumerate(pdf_files[:5], 1):
    print(f"{i}. {os.path.basename(file)}")

현재 작업 디렉토리: /home/sohee/Bidding-Mate
✅ 총 PDF 파일 개수: 30

처음 5개 파일:
1. 국방과학연구소_대용량 자료전송시스템 고도화.pdf
2. 대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.pdf
3. 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
4. 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
5. 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf


In [15]:
from langchain_community.document_loaders import PyPDFLoader
# PDF 파일 열어서 텍스트 추출하는 라이브러리
import os

# 테스트: 첫 번째 PDF 파일만 먼저 로드해보기
# (30개 전부 하기 전에 1개로 테스트!)
test_pdf = pdf_files[0]
print(f"📄 테스트 파일: {os.path.basename(test_pdf)}")

# PDF 로더 생성 및 로드
# PyPDFLoader는 PDF를 페이지별로 읽어서 Document 객체로 만들어줌
loader = PyPDFLoader(test_pdf)
documents = loader.load()
# loader.load()는 실제로 PDF를 읽어서 페이지별로 Document 객체를 만들어줌

# 결과 확인
print(f"\n✅ 총 {len(documents)} 페이지 로드 완료")
print(f"📊 전체 텍스트 길이: {sum(len(doc.page_content) for doc in documents):,}자")

# 첫 페이지 내용 미리보기 (처음 500자만)
print(f"\n📖 첫 페이지 내용 미리보기 (500자):")
print(documents[0].page_content[:500])
# page_content는 실제 텍스트의 내용
print("...")

📄 테스트 파일: 국방과학연구소_대용량 자료전송시스템 고도화.pdf

✅ 총 87 페이지 로드 완료
📊 전체 텍스트 길이: 55,548자

📖 첫 페이지 내용 미리보기 (500자):
제 안 요 청 서
사 업 명대용량 자료전송시스템 고도화주관기관
2024.07.담 당소 속 Tel.계 약재 무 실  내 자 팀0 4 2 - 8 2 1 - 3 8 2 7실 무정 보 화 기 술 실  정 보 보 호 팀0 4 2 - 8 2 1 - 2 4 6 1※ 본 자료는 제안 내용의 설명을 위한 배포자료로, 이외의 목적으로 무단복제,    전달 및 사용하는 행위를 일절 금함.
...


In [16]:
from langchain_community.document_loaders import PyPDFLoader
from tqdm import tqdm  # 진행 상황 표시용
import time

# 모든 PDF 문서를 담을 리스트
all_documents = []

print(f"📚 총 {len(pdf_files)}개 PDF 파일 로딩 시작!\n")

# 30개 파일 하나씩 로드
for i, pdf_path in enumerate(pdf_files, 1):
    try:
        # PDF 로더 생성
        loader = PyPDFLoader(pdf_path)
        
        # 문서 로드
        docs = loader.load()
        
        # 전체 리스트에 추가
        all_documents.extend(docs)
        
        # 진행 상황 출력
        total_chars = sum(len(doc.page_content) for doc in docs)
        print(f"[{i}/30] ✅ {os.path.basename(pdf_path)}")
        print(f"        └─ {len(docs)}페이지, {total_chars:,}자")
        
    except Exception as e:
        print(f"[{i}/30] ❌ {os.path.basename(pdf_path)} - 오류: {e}")
        continue

# 최종 결과
print(f"\n" + "="*60)
print(f"🎉 로딩 완료!")
print(f"📊 총 문서 수: {len(all_documents)}개 (페이지)")
print(f"📝 총 텍스트 길이: {sum(len(doc.page_content) for doc in all_documents):,}자")
print(f"📈 평균 페이지당 텍스트: {sum(len(doc.page_content) for doc in all_documents) // len(all_documents):,}자")
print("="*60)

📚 총 30개 PDF 파일 로딩 시작!

[1/30] ✅ 국방과학연구소_대용량 자료전송시스템 고도화.pdf
        └─ 87페이지, 55,548자
[2/30] ✅ 대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.pdf
        └─ 72페이지, 64,986자
[3/30] ✅ 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
        └─ 63페이지, 45,903자
[4/30] ✅ 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
        └─ 113페이지, 72,051자
[5/30] ✅ 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf
        └─ 37페이지, 61,297자
[6/30] ✅ BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
        └─ 48페이지, 74,922자
[7/30] ✅ 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
        └─ 86페이지, 60,235자
[8/30] ✅ 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
        └─ 49페이지, 42,680자
[9/30] ✅ 국민연금공단_2024년 이러닝시스템 운영 용역.pdf
        └─ 68페이지, 62,690자
[10/30] ✅ 광주과학기술원_학사시스템 기능개선 사업.pdf
        └─ 33페이지, 57,738자
[11/30] ✅ 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf
        └─ 48페이지, 54,807자
[12/30] ✅ 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.pdf
        └─ 41페이지, 68,571자
[13/30] ✅ 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
        └─ 118페이지, 83,471자
[14/30] 

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✂️ 텍스트 청킹 시작!\n")

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 하나당 [1]1000자
    chunk_overlap=200,      # 앞뒤 청크가 200자[1]씩 겹치게
    length_function=len,    # 길이 측정 방식
    separators=["\n\n", "\n", " ", ""]  # 구분자 우선순위
)

# 실제 청킹 실행
chunks = text_splitter.split_documents(all_documents)

# 결과 확인
print(f"✅ 청킹 완료!")
print(f"📊 원본: {len(all_documents)}개 페이지")
print(f"📊 청크: {len(chunks)}개 조각")
print(f"📈 비율: 페이지 1개 → 약 {len(chunks) / len(all_documents):.1f}개 청크")

# 첫 번째 청크 샘플 확인
print(f"\n📖 첫 번째 청크 샘플 (처음 500자):")
print(chunks[0].page_content[:500])
print("...")

print(f"\n📝 청크 메타데이터 (어떤 파일의 몇 페이지인지):")
print(chunks[0].metadata)

✂️ 텍스트 청킹 시작!

✅ 청킹 완료!
📊 원본: 2353개 페이지
📊 청크: 3642개 조각
📈 비율: 페이지 1개 → 약 1.5개 청크

📖 첫 번째 청크 샘플 (처음 500자):
제 안 요 청 서
사 업 명대용량 자료전송시스템 고도화주관기관
2024.07.담 당소 속 Tel.계 약재 무 실  내 자 팀0 4 2 - 8 2 1 - 3 8 2 7실 무정 보 화 기 술 실  정 보 보 호 팀0 4 2 - 8 2 1 - 2 4 6 1※ 본 자료는 제안 내용의 설명을 위한 배포자료로, 이외의 목적으로 무단복제,    전달 및 사용하는 행위를 일절 금함.
...

📝 청크 메타데이터 (어떤 파일의 몇 페이지인지):
{'producer': 'Hancom PDF 1.3.0.515', 'creator': '한컴오피스 한글 2018', 'creationdate': '2025-12-18T11:40:14+09:00', 'author': 'user', 'moddate': '2025-12-18T11:40:14+09:00', 'pdfversion': '1.4', 'source': 'team_members/sohee/sample_pdfs/국방과학연구소_대용량 자료전송시스템 고도화.pdf', 'total_pages': 87, 'page': 0, 'page_label': '1'}


In [18]:
# 이제 임베딩이랑 VectorDB

import os

# API 키 설정 (실제 키로 변경!)

# print("✅ API 키 설정 완료")

# 그냥 내 개인 API key로 한 번 해보자... 자꾸 PermissionDenyError 뜨니까 답답ㅎ해
# API key 일단 지움 git push 때문에

✅ API 키 설정 완료


In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import time
from tqdm import tqdm

print("🔢 임베딩 생성 및 VectorDB 저장 시작!\n")

# OpenAI 임베딩 모델 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # 빠르고 저렴한 모델
)

# VectorDB 저장 경로
persist_directory = "team_members/sohee/vectordb"

# ChromaDB에 청크들 저장 (임베딩 자동 생성됨!)
print(f"💾 {len(chunks)}개 청크를 VectorDB에 저장 중...")
print("⏰ 약 2~3분 소요 예상...\n")

start_time = time.time()

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

elapsed_time = time.time() - start_time

print(f"\n✅ VectorDB 저장 완료!")
print(f"⏱️ 소요 시간: {elapsed_time:.1f}초")
print(f"💾 저장 경로: {persist_directory}")
print(f"📊 저장된 청크 수: {vectordb._collection.count()}개")

🔢 임베딩 생성 및 VectorDB 저장 시작!

💾 3642개 청크를 VectorDB에 저장 중...
⏰ 약 2~3분 소요 예상...


✅ VectorDB 저장 완료!
⏱️ 소요 시간: 45.2초
💾 저장 경로: team_members/sohee/vectordb
📊 저장된 청크 수: 9732개


In [20]:
print("🔍 VectorDB 검색 테스트 시작!\n")

# 테스트 질문
test_query = "사업 예산이 얼마인가요?"

print(f"❓ 질문: {test_query}\n")

# VectorDB에서 유사한 청크 검색
# similarity_search: 질문과 가장 비슷한 청크들을 찾아줌
results = vectordb.similarity_search(
    test_query,
    k=3  # 상위 3개 결과만 가져오기
)

print(f"✅ 관련 문서 {len(results)}개 발견!\n")
print("="*60)

# 검색 결과 출력
for i, doc in enumerate(results, 1):
    print(f"\n[검색 결과 {i}]")
    print(f"📄 출처: {doc.metadata['source'].split('/')[-1]}")  # 파일명
    print(f"📖 페이지: {doc.metadata['page'] + 1}")  # 페이지 번호
    print(f"📝 내용 (처음 300자):")
    print(doc.page_content[:300])
    print("...")
    print("-"*60)

🔍 VectorDB 검색 테스트 시작!

❓ 질문: 사업 예산이 얼마인가요?

✅ 관련 문서 3개 발견!


[검색 결과 1]
📄 출처: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
📖 페이지: 55
📝 내용 (처음 300자):
목     차 1. 사 업 명 ················································································································ 2. 사업기간 ················································································································ 3. 사업목적 ·········································
...
------------------------------------------------------------

[검색 결과 2]
📄 출처: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf
📖 페이지: 55
📝 내용 (처음 300자):
목     차 1. 사 업 명 ················································································································ 2. 사업기간 ················································································································ 3. 사업목적 ·········································
...
------------------------------------------------------------

[검색 결과 3]
📄 출처: 국민연금공단_사업장 사회보험료 

In [21]:
%pip install -U langchain langchain-community langchain-core

Note: you may need to restart the kernel to use updated packages.


In [23]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"
# API key 있으면 git push가 안 된다네요..! 개인 키나 공용 키 사용 부탁드립니다.

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

print("🤖 간단한 RAG 시스템 구축!\n")

# GPT 모델
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "당신은 입찰공고 전문가입니다.\n"
     "아래 문서들을 참고하여 질문에 답변하세요.\n"
     "답을 모르면 모른다고 말하세요.\n\n"
     "참고 문서:\n{context}"),
    ("human", "{question}")
])

def ask_question(question):
    """RAG 시스템으로 질문하기"""
    
    # 1. VectorDB에서 관련 문서 검색
    docs = vectordb.similarity_search(question, k=5)
    
    # 2. 문서들을 하나의 텍스트로 합치기
    context = "\n\n---\n\n".join([
        f"[문서 {i+1}] {doc.metadata['source'].split('/')[-1]} (p.{doc.metadata['page']+1})\n{doc.page_content}"
        for i, doc in enumerate(docs)
    ])
    
    # 3. 프롬프트 생성
    messages = prompt_template.format_messages(
        context=context,
        question=question
    )
    
    # 4. LLM에게 질문
    response = llm.invoke(messages)
    
    # 5. 결과 출력
    print(f"❓ 질문: {question}\n")
    print(f"💬 답변:\n{response.content}\n")
    print(f"📚 참조 문서:")
    for i, doc in enumerate(docs[:3], 1):
        print(f"  {i}. {doc.metadata['source'].split('/')[-1]} (p.{doc.metadata['page']+1})")
    print("="*60)
    
    return response.content

print("✅ RAG 시스템 준비 완료!\n")
print("="*60)

# 테스트 질문들
test_questions = [
    "사업 예산이 얼마인가요?",
    "제출 기한은 언제인가요?",
    "어떤 서류를 제출해야 하나요?"
]

for question in test_questions:
    ask_question(question)
    print()

🤖 간단한 RAG 시스템 구축!

✅ RAG 시스템 준비 완료!

❓ 질문: 사업 예산이 얼마인가요?

💬 답변:
2024년 벤처확인종합관리시스템 기능 고도화 용역사업의 소요예산은 352,000,000원(부가가치세 포함)입니다.

📚 참조 문서:
  1. 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf (p.55)
  2. 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.pdf (p.55)
  3. 국민연금공단_사업장 사회보험료 지원 고시 개정에 따른 정보시스템 보.pdf (p.53)

❓ 질문: 제출 기한은 언제인가요?

💬 답변:
제출 기한에 대한 정보는 문서에 명시된 공고문을 참조해야 합니다. 각 입찰 공고마다 제출 기한이 다를 수 있으므로, 해당 공고문을 확인하시기 바랍니다.

📚 참조 문서:
  1. 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf (p.1)
  2. 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.pdf (p.1)
  3. 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf (p.59)

❓ 질문: 어떤 서류를 제출해야 하나요?

💬 답변:
제출해야 하는 서류는 다음과 같습니다:

1. **결과보고서**: 서술식으로 작성된 연구보고서를 한글(HWP) 형태로 제출해야 합니다.
2. **연구데이터**: 연구 진행 과정에서 산출된 데이터(설문 로데이터(raw data), 가공데이터, 심층 인터뷰 녹취록, 자문회의록 등)를 제출해야 합니다.
3. **과업 종료 시 수집 자료**: USB, 외장하드 등 저장매체에 모든 자료를 저장하여 제출해야 하며, 검수보고서 5부와 과업 추진과 관련된 모든 자료가 저장된 저장매체 3개를 제출해야 합니다.
4. **투입인력 증빙서류**: 계약 체결 시까지 해당 참여 인력의 자격, 경력, 유사사업 참여 실적을 상세히 제시한 증빙서류를 제출해야 합니다.

추가적으로, 성과품 제출 이후